In [6]:
#Relation extraction using SEAL
seeds = []
seedsLocation = "data/relationSeeds"
with open(seedsLocation) as f:
    seeds = [line.strip() for line in f]
    
relationSeeds = []
for seed in seeds:
    tokens = seed.split("\t")
    if len(tokens)<=1:
        continue
    arg1 = tokens[0].strip()
    arg2 = tokens[1].strip()
    relationSeeds.append((arg1, arg2))
print("seeds are: ")
for seed in relationSeeds:
    print(seed)

seeds are: 
('OS', 'Android')
('RAM', '2 GB')
('Item Weight', '150 g')
('Weight', '150 Grams')


In [ ]:
def getClosestPairs(l1, l2):
    r = set([])
    for i1 in l1:
        for i2 in l2:
            if i2>i1 and i2-i1<100 and not (i1, i2) in r:
                r.add((i1, i2))
            elif i2<i1 and i1-i2<100 and not (i2, i1) in r:
                r.add((i2, i1))
    return r

In [41]:
def doIntersection(l):
    if len(l)==1:
        return []
    s1 = l[0]
    s2 = l[1:]
    result = []
    for s in s2:
        r = s1.intersection(s)
        result.extend(r)
    result.extend(doIntersection(s2))
    return list(set(result))
    
def getStringFromTagEnd(s):
#     s = reversed(s)
    result = ""
    for c in reversed(s):
        if c ==">":
            break
        result+=str(c)
    if len(result)>100:
        return ""
    return result[::-1]


def getStringTillTagStart(s):
    result = ""
    for c in s:
        if c=="<":
            break
        result+=str(c)
    if len(result)>100:
        return ""
    return result

In [43]:
import os
import re
ecommerceFile = "data/Lenovo Vibe K5.html"
ecommerceData = ""
with open(ecommerceFile) as f:
    ecommerceData = f.read()
ecommerceData = re.sub( '\s+', ' ', ecommerceData).strip()


presentSeeds = []
for (arg1, arg2) in relationSeeds:
    arg1Location = ecommerceData.find(arg1)
    arg2Location = ecommerceData.find(arg2)
    if arg1Location == -1 or arg2Location==-1:
        continue
    else:
        presentSeeds.append((arg1, arg2))
for seed in presentSeeds:
    print(seed)

    
contexts = []
for (arg1, arg2) in relationSeeds:
    arg1Contexts = []
    for m in re.finditer(arg1, ecommerceData):
        start = m.start()
        end   = m.end()
        arg1Contexts.append(end+1)
    arg2Contexts = []
    for m in re.finditer(arg2, ecommerceData):
        start = m.start()
        end   = m.end()
        arg2Contexts.append(start-1)
    closestPairs = getClosestPairs(arg1Contexts, arg2Contexts)
    contextsPerSeed = set([])
    for (a, b) in closestPairs:
        contextsPerSeed.add(ecommerceData[a:b])
    contexts.append(contextsPerSeed)
    
    
    
contexts = doIntersection(contexts)
for context in contexts:
    print("Context is ")
    print(context)
    
contextLocs = []
foundRelations = []
for context in contexts:
    for m in re.finditer(context, ecommerceData):
        start  = m.start()
        end    = m.end()
        arg1   = getStringFromTagEnd(ecommerceData[:start]).strip()
        arg2   = getStringTillTagStart(ecommerceData[end+1:]).strip()
        if len(arg1)>0 and len(arg2)>0:
            foundRelations.append((arg1, arg2))
            
            
print("Found relations are")
for relation in foundRelations:
    print(relation)

    


('OS', 'Android')
('RAM', '2 GB')
('Item Weight', '150 g')
('Weight', '150 Grams')
Context is 
</td><td class="value"
Found relations are
('OS', 'Android')
('RAM', '2 GB')
('Item Weight', '150 g')
('Product Dimensions', '14.2 x 0.8 x 7.1 cm')
('Item model number', 'PA2M0111IN')
('Wireless communication technologies', 'Bluetooth, WiFi Hotspot')
('Other camera features', '5MP')
('Form factor', 'Touchscreen Phone')
('Weight', '150 Grams')
('Colour', 'Grey')
('Battery Power Rating', '2750')
('Phone Talk Time', '15.1 Hours')
('Phone Standby Time (with data)', '322 Hours')
('Whats in the box', 'Handset, Battery, Charger and User Manual')
('ASIN', 'B01M07568E')
('Best Sellers Rank', '#48 in Electronics (')
('Date First Available', '1 January 2017')


In [40]:
"sanjeev"[::-1]

'veejnas'

In [ ]:
#for particular set represented as list of strings find common prefix between them
def createAllPrefix (s, l):
    ansList = []
    if len(l)==1:
        for ch in l[0]:
            s = s+ch 
            ansList.append(s.strip())
        return list(set(ansList))
    dictMap = {}
    for item in l:
        ch = item[0]
        if ch in dictMap:
            dictMap[ch].append(item[1:])
        else:
            dictMap[ch] = [item[1:]]
    for k, v in dictMap.items():
        ansList.append((s+k).strip())
        nextAnswer = createAllPrefix(s+k, v)
        ansList.extend(nextAnswer)
    return list(set(ansList))

def createPossibleStrings(l):
    return set(createAllPrefix ("", l))

createAllPrefix("", ["sanjeev", "kumar"])    